In [ ]:
#pip install sidrapy

In [127]:
import sidrapy
import pandas as pd
import requests
import numpy as np

In [136]:
#na página do ibge você pode clicar em gerar link para visualizar quais parâmetros devem ser inseridos aqui
#mas não é intuitivo, tem que seguir essa documentação: https://apisidra.ibge.gov.br/home/ajuda

empresas = sidrapy.table.get_table(
    table_code="2718",
    territorial_level="1",
    ibge_territorial_code="all",
    period="last 9",
    variable="630",
    classifications={"12762": "all", "370": "9504", "369": "all"},
    
)

empresas.columns = empresas.iloc[0] #troca a primeira linha para o nome da coluna
empresas = empresas.iloc[1:, :] #retira a primeira obs. que virou o nome da coluna

#Essa tabela tem todas as info. possíveis. Vamos gerar tabelas específicas abaixo:

In [137]:
empresas = empresas.drop(columns = ['Nível Territorial', 'Unidade de Medida', 'Nível Territorial (Código)', 'Unidade de Medida (Código)',
                                    'Brasil (Código)', 'Brasil', 'Variável (Código)', 
                                    'Classificação Nacional de Atividades Econômicas (CNAE 2.0) (Código)',
                                    'Faixas de pessoal ocupado assalariado (Código)',
                                    'Faixas de pessoal ocupado assalariado',
                                    'Tipo de evento da empresa (Código)',
                                    'Ano (Código)',
                                   'Variável'])

In [138]:
empresas = empresas.rename(columns={"Classificação Nacional de Atividades Econômicas (CNAE 2.0)": "cnae_secao",
                                     "Tipo de evento da empresa": "evento",
                                    "Valor": "qtde_empresas"}) 

In [139]:
empresas['qtde_empresas'] = empresas['qtde_empresas'].replace("-", np.nan)

In [140]:
empresas['qtde_empresas'] = empresas['qtde_empresas'].astype("float")

In [171]:
empresas_pivot = empresas.pivot_table(index=['Ano', 'cnae_secao'], 
                                      columns='evento', 
                                      values='qtde_empresas').reset_index()

In [172]:
empresas_pivot.head(5)

evento,Ano,cnae_secao,Entrada,Nascimento,Reentrada,Saída de atividade,Sobrevivente,Total de empresas ativas
0,2012,"A Agricultura, pecuária, produção florestal, p...",7100.0,4898.0,2202.0,5849.0,26458.0,33558.0
1,2012,B Indústrias extrativas,2005.0,1248.0,757.0,1618.0,8638.0,10643.0
2,2012,C Indústrias de transformação,65094.0,45640.0,19454.0,64606.0,370890.0,435984.0
3,2012,D Eletricidade e gás,595.0,440.0,155.0,378.0,1697.0,2292.0
4,2012,"E Água, esgoto, atividades de gestão de resídu...",2031.0,1473.0,558.0,1530.0,7761.0,9792.0


In [173]:
empresas_pivot['cnae_secao'] = empresas_pivot['cnae_secao'].str.slice(start=2)

In [174]:
empresas_pivot.head(5)

evento,Ano,cnae_secao,Entrada,Nascimento,Reentrada,Saída de atividade,Sobrevivente,Total de empresas ativas
0,2012,"Agricultura, pecuária, produção florestal, pes...",7100.0,4898.0,2202.0,5849.0,26458.0,33558.0
1,2012,Indústrias extrativas,2005.0,1248.0,757.0,1618.0,8638.0,10643.0
2,2012,Indústrias de transformação,65094.0,45640.0,19454.0,64606.0,370890.0,435984.0
3,2012,Eletricidade e gás,595.0,440.0,155.0,378.0,1697.0,2292.0
4,2012,"Água, esgoto, atividades de gestão de resíduos...",2031.0,1473.0,558.0,1530.0,7761.0,9792.0


In [175]:
empresas_pivot.to_csv("informacoes_empresas.csv")

In [176]:
empresas_pivot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186 entries, 0 to 185
Data columns (total 8 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Ano                       186 non-null    object 
 1   cnae_secao                186 non-null    object 
 2   Entrada                   184 non-null    float64
 3   Nascimento                184 non-null    float64
 4   Reentrada                 180 non-null    float64
 5   Saída de atividade        184 non-null    float64
 6   Sobrevivente              184 non-null    float64
 7   Total de empresas ativas  185 non-null    float64
dtypes: float64(6), object(2)
memory usage: 11.8+ KB
